In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/954471/954471/Test/M/121160480_3678313258869851_5818106878740531607_o.jpg
/kaggle/input/954471/954471/Test/M/121623008_2688765031372476_1265210388791973144_n.jpg
/kaggle/input/954471/954471/Test/M/121558243_3459617724076248_4826110803681428223_n.jpg
/kaggle/input/954471/954471/Test/M/121218785_2888174644739280_5191976153731649311_n.jpg
/kaggle/input/954471/954471/Test/F/121638990_1012059519260274_2330212418373086362_o.jpg
/kaggle/input/954471/954471/Test/F/121558240_972714016543857_6033716185027836421_o.jpg
/kaggle/input/954471/954471/Test/F/121711378_1739798592838691_2434137178508018006_n.jpg
/kaggle/input/954471/954471/Test/F/121297902_1739796202838930_2202079621057532442_n.jpg
/kaggle/input/954471/954471/Train/M/121381660_3348014425282334_9154920501988623614_o.jpg
/kaggle/input/954471/954471/Train/M/121543339_3481304788558895_349846518547631903_n.jpg
/kaggle/input/954471/954471/Train/M/121368482_3314461121964215_543249990768154260_o.jpg
/kaggle/input/954471/954471/Trai

In [4]:
test_dir="/kaggle/input/954471/954471/Test"
train_dir="/kaggle/input/954471/954471/Train"

train_dir_F = train_dir + '/F'
train_dir_M = train_dir + '/M'
test_dir_F = test_dir + '/F'
test_dir_M = test_dir + '/M'

In [5]:
print('number of F training images - ',len(os.listdir(train_dir_F)))
print('number of M training images - ',len(os.listdir(train_dir_M)))
print('number of F testing images - ',len(os.listdir(test_dir_F)))
print('number of M testing images - ',len(os.listdir(test_dir_M)))

number of F training images -  8
number of M training images -  8
number of F testing images -  4
number of M testing images -  4


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        rescale = 1.0/255.0,   # Intensity Normalized ให้เหลือ 0-1
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)ภาพบิดอัตโนมัติ
        zoom_range = 0.2, # Randomly zoom image 
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,   # randomly flip images
        validation_split=0.2)

In [7]:
batch_size = 2
img_height = 64 
img_width  = 64 
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = data_generator.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

Found 14 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'F': 0, 'M': 1}

In [121]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
# preparing the layers in the Convolutional Deep Neural Network
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', input_shape = train_generator.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = train_generator.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

#แปลงเป็น vector
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
# model.add(Dropout(rate = 0.3))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.2))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [122]:
fitted_model = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 200)

Epoch 1/200
7/7 [==============================] - 1s 92ms/step - loss: 0.7042 - accuracy: 0.5000 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 2/200
7/7 [==============================] - 0s 18ms/step - loss: 0.7057 - accuracy: 0.3571 - val_loss: 0.6949 - val_accuracy: 0.5000
Epoch 3/200
7/7 [==============================] - 0s 18ms/step - loss: 0.7051 - accuracy: 0.4286 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 4/200
7/7 [==============================] - 0s 18ms/step - loss: 0.6824 - accuracy: 0.5714 - val_loss: 0.6936 - val_accuracy: 0.5000
Epoch 5/200
7/7 [==============================] - 0s 18ms/step - loss: 0.6939 - accuracy: 0.4286 - val_loss: 0.6914 - val_accuracy: 0.5000
Epoch 6/200
7/7 [==============================] - 0s 19ms/step - loss: 0.6930 - accuracy: 0.5000 - val_loss: 0.6920 - val_accuracy: 0.5000
Epoch 7/200
7/7 [==============================] - 0s 17ms/step - loss: 0.6943 - accuracy: 0.5714 - val_loss: 0.6864 - val_accuracy: 0.5000
Epoch 8/200
7/7 [===

7/7 [==============================] - 0s 18ms/step - loss: 0.0157 - accuracy: 1.0000 - val_loss: 0.8304 - val_accuracy: 0.5000
Epoch 59/200
7/7 [==============================] - 0s 18ms/step - loss: 1.7316e-04 - accuracy: 1.0000 - val_loss: 0.0773 - val_accuracy: 1.0000
Epoch 60/200
7/7 [==============================] - 0s 19ms/step - loss: 0.3698 - accuracy: 0.9286 - val_loss: 0.0405 - val_accuracy: 1.0000
Epoch 61/200
7/7 [==============================] - 0s 19ms/step - loss: 0.0179 - accuracy: 1.0000 - val_loss: 0.0482 - val_accuracy: 1.0000
Epoch 62/200
7/7 [==============================] - 0s 18ms/step - loss: 0.0235 - accuracy: 1.0000 - val_loss: 3.6420e-04 - val_accuracy: 1.0000
Epoch 63/200
7/7 [==============================] - 0s 19ms/step - loss: 0.0173 - accuracy: 1.0000 - val_loss: 5.0740e-04 - val_accuracy: 1.0000
Epoch 64/200
7/7 [==============================] - 0s 17ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.0742 - val_accuracy: 1.0000
Epoch 65/200
7

7/7 [==============================] - 0s 22ms/step - loss: 1.8144e-05 - accuracy: 1.0000 - val_loss: 5.4373e-09 - val_accuracy: 1.0000
Epoch 114/200
7/7 [==============================] - 0s 22ms/step - loss: 8.0544e-05 - accuracy: 1.0000 - val_loss: 2.5776e-07 - val_accuracy: 1.0000
Epoch 115/200
7/7 [==============================] - 0s 19ms/step - loss: 2.0451e-05 - accuracy: 1.0000 - val_loss: 2.8234e-08 - val_accuracy: 1.0000
Epoch 116/200
7/7 [==============================] - 0s 19ms/step - loss: 7.0543e-06 - accuracy: 1.0000 - val_loss: 6.5175e-09 - val_accuracy: 1.0000
Epoch 117/200
7/7 [==============================] - 0s 27ms/step - loss: 7.0437e-07 - accuracy: 1.0000 - val_loss: 8.2879e-06 - val_accuracy: 1.0000
Epoch 118/200
7/7 [==============================] - 0s 24ms/step - loss: 2.8039e-05 - accuracy: 1.0000 - val_loss: 7.5900e-08 - val_accuracy: 1.0000
Epoch 119/200
7/7 [==============================] - 0s 23ms/step - loss: 2.4550e-05 - accuracy: 1.0000 - val_loss

Epoch 168/200
7/7 [==============================] - 0s 18ms/step - loss: 5.4246e-06 - accuracy: 1.0000 - val_loss: 3.7885e-09 - val_accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 18ms/step - loss: 3.4461e-04 - accuracy: 1.0000 - val_loss: 2.7532e-08 - val_accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 18ms/step - loss: 1.3506e-04 - accuracy: 1.0000 - val_loss: 2.8214e-07 - val_accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 18ms/step - loss: 7.1214e-06 - accuracy: 1.0000 - val_loss: 8.5741e-08 - val_accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 18ms/step - loss: 1.7120e-05 - accuracy: 1.0000 - val_loss: 3.1248e-08 - val_accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 19ms/step - loss: 5.3050e-07 - accuracy: 1.0000 - val_loss: 1.9659e-06 - val_accuracy: 1.0000
Epoch 174/200
7/7 [==============================] - 0s 19ms/step - loss: 3.5241e-05 - accuracy: 1.0

In [144]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/M/121623008_2688765031372476_1265210388791973144_n.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))

[[1.]]
